In [ ]:
SRC='/home/jmurga/mkt/201903/scripts/src'
DATA='/home/jmurga/mkt/201903/rawData/humans'

In [ ]:
import os
import re
import sys
import allel
import numpy as np
import pandas as pd
import pyfaidx as px

In [ ]:
sys.path.insert(0, SRC)
from reverseComplement import reverseComplement
from degenerancy import degenerate

### Human genes

To execute bash code we created a snippets through nbextensions containing the following paths in order to avoid copy and paste cells. %%bash magic do not recognize previous variables. Adding mkdir command to create necesary paths

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201903/rawData/humans/annotations'
BASIC='/home/jmurga/mkt/201903/rawData/humans/annotations/basicAnnotation'
CDS='/home/jmurga/mkt/201903/rawData/humans/annotations/cds'
GENES='/home/jmurga/mkt/201903/rawData/humans/annotations/genes'
TEMPORAL='/home/jmurga/mkt/201903/rawData/humans/annotations/tmp'
ALLELEFREQ='/home/jmurga/mkt/201903/rawData/humans/alleleFrequencies'

mkdir -p ${DATA}
mkdir -p ${BASIC}
mkdir -p ${CDS}
mkdir -p ${GENES}
mkdir -p ${TEMPORAL}
mkdir -p ${ALLELEFREQ}

#### Download [gencode annotation](https://www.gencodegenes.org/human/release_27lift37.html)

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201903/rawData/humans'
ANNOTATIONS='/home/jmurga/mkt/201903/rawData/humans/annotations'
EXPRESSION='/home/jmurga/mkt/201903/rawData/humans/expression'

cd ${ANNOTATIONS}
# Gencode annotation
wget ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_27/GRCh37_mapping/gencode.v27lift37.basic.annotation.gff3.gz
gunzip *.gz

#### Download reference fastas

In [ ]:
%%bash 
DATA='/home/jmurga/mkt/201903/rawData/humans'
FASTAS='/home/jmurga/mkt/201903/rawData/humans/fastas/'
mkdir - p ${FASTAS}
mkdir - p ${FASTAS}/ref

cd ${FASTAS}/ref

wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/human_g1k_v37.fasta.gz 
wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/human_g1k_v37.fasta.fai

In [ ]:
file = px.Fasta('/home/jmurga/mkt/201903/rawData/humans/fastas/ref/human_g1k_v37.fasta',duplicate_action='first',sequence_always_upper=True,read_long_names=True)

samples = list(file.keys())[0:24]

for nchr in range(0,len(samples),1):
    print('>chr' + samples[nchr].split(' ')[0])
    f = open('/home/jmurga/mkt/201903/rawData/humans/fastas/ref/chr' + samples[nchr].split(' ')[0] + '.fa' , 'w')
    tmp = file[samples[nchr]][:].seq
    f.write('>chr' + samples[nchr].split(' ')[0] + '\n' + file[samples[nchr]][:].seq)
    f.close()

In [ ]:
%%bash 
rm ${FASTA}/ref/human_g1k*

#### Parsing and cleaning gencode annotation to execute all operations by chr
Operate by chr is faster due to grep on smaller files. Each folder contain an specific file foreach chromosome. 

In [ ]:
%%bash
ANNOTATIONS='/home/jmurga/mkt/201903/rawData/humans/annotations'
BASIC='/home/jmurga/mkt/201903/rawData/humans/annotations/basicAnnotation'
CDS='/home/jmurga/mkt/201903/rawData/humans/annotations/cds'
GENES='/home/jmurga/mkt/201903/rawData/humans/annotations/genes'

mkdir -p ${BASIC}
mkdir -p ${CDS}
mkdir -p ${GENES}

# Deleting commented lines
sed -i '/^#/ d' ${ANNOTATIONS}/gencode.v27lift37.basic.annotation.gff3
# Extract genes information
grep -P "\tgene\t" ${ANNOTATIONS}/gencode.v27lift37.basic.annotation.gff3 | grep -vP "chrM\t" > ${ANNOTATIONS}/gencode.v27lift37.genes.gff3
# Extract CDS information. Only protein coding genes
grep -P "\tCDS\t" ${ANNOTATIONS}/gencode.v27lift37.basic.annotation.gff3 | grep -vP "chrM\t" > ${ANNOTATIONS}/gencode.v27lift37.cds.gff3
# Coding gene list. gene_id always on column 9, 3th field
cut -f1,9 gencode.v27lift37.cds.gff3 | tr ';' '\t' | cut -f1,4 | sort -u | sort -k1,1 > ${ANNOTATIONS}/codingGeneList.txt

CHR=( 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 X Y )
# Parse gff file by chr
for chrNumber in "${CHR[@]}"
do
    echo ${chrNumber}
    grep -P "chr${chrNumber}\t"  ${ANNOTATIONS}/gencode.v27lift37.basic.annotation.gff3 | sort -k4,4n > ${BASIC}/gencode.v27lift37.basic.annotation.chr${chrNumber}.gff3 
    grep -P "chr${chrNumber}\t"  ${ANNOTATIONS}/gencode.v27lift37.cds.gff3 | sort -k1,1 -k4,4n > ${CDS}/gencode.v27lift37.cds.chr${chrNumber}.gff3 
    grep -P "chr${chrNumber}\t"  ${ANNOTATIONS}/gencode.v27lift37.genes.gff3 | sort -k1,1 -k4,4n > ${GENES}/gencode.v27lift37.genes.chr${chrNumber}.gff3 
done

rm ${ANNOTATIONS}/gencode.v27lift37.basic.annotation.gff3

#### Basic cleaned gene file
This file will include information about chromosomes, start coordinates, end coordinates, strand, gene id and gene name. It will be and perform calculations on gene coordinates. Kind of gff file easier to work with

In [ ]:
%%bash
ANNOTATIONS='/home/jmurga/mkt/201903/rawData/humans/annotations'
BASIC='/home/jmurga/mkt/201903/rawData/humans/annotations/basicAnnotation'
GENES='/home/jmurga/mkt/201903/rawData/humans/annotations/genes'
# Cleaned gene file. Including chr, start, end, id and symbol. At gene entries geneid and genesymbol always on field 5 and 7or8
touch ${ANNOTATIONS}/gencodeGenesCleaned.tab 
printf "chr\tstart\tend\tstrand\tid\tname\n" > ${ANNOTATIONS}/gencodeGenesCleaned.tab

time while read LINE;
do 
    CHR=$(echo ${LINE} | cut -d' ' -f1 ); 
    GENE=$(echo ${LINE} | cut -d' ' -f2); 
    fgrep `echo ${GENE}`  ${GENES}/gencode.v27lift37.genes.${CHR}.gff3 | grep -P "\tgene\t" | cut -f1,4,5,7,9 | tr ';' '\t' | cut -f1,2,3,4,6,8,9 | awk '{if($6 ~ /gene_name/) print $1,$2,$3,$4,$5,$6;else print $1,$2,$3,$4,$5,$7}' | tr ' ' '\t' | sed 's/gene_id=//g' | sed 's/gene_name=//g' 
done < ${ANNOTATIONS}/codingGeneList.txt >> ${ANNOTATIONS}/gencodeGenesCleaned.tab

sort -k1,1 -k2,2n ${ANNOTATIONS}/gencodeGenesCleaned.tab > ${ANNOTATIONS}/tmpAnnotation && mv ${ANNOTATIONS}/tmpAnnotation ${ANNOTATIONS}/gencodeGenesCleaned.tab

In [ ]:
dfGenes = pd.read_csv(DATA + '/annotations/gencodeGenesCleaned.tab',header = 0,sep='\t')
dfGenes.head(10)

In [ ]:
inputGeneList = dfGenes.loc[:,'id'].apply(lambda x: re.sub(r"\..*$", "",x))
idCleaned = pd.DataFrame(data = {'id': dfGenes.loc[:,'id'],'idCleaned':inputGeneList})
idCleaned.head(10)

In [ ]:
idCleaned.to_csv(DATA + '/annotations/idCleaned.tab',header=True,index=False,sep='\t')

### CDS human degenerancy 

#### Raw cds coordinates

In [ ]:
%%bash
ANNOTATIONS='/home/jmurga/mkt/201903/rawData/humans/annotations'
CDS='/home/jmurga/mkt/201903/rawData/humans/annotations/cds'
TEMPORAL='/home/jmurga/mkt/201903/rawData/humans/annotations/tmp'

touch ${ANNOTATIONS}/cdsCoordinates.tab
printf "id\tchr\ttranscript\ttranscriptSize\tcoordinates\n" > ${ANNOTATIONS}/cdsCoordinates.tab

time tail -n+2 ${ANNOTATIONS}/gencodeGenesCleaned.tab | while read LINE;
do 
    echo '*************'
    echo $LINE | cut -d' ' -f5
    
    CHR=$(echo ${LINE} | cut -d' ' -f1)
    GENE=$(echo ${LINE} | cut -d' ' -f5)

    fgrep ${GENE} ${CDS}/gencode.v27lift37.cds.${CHR}.gff3  | cut -f9 | tr ';' '\n' | fgrep transcript_id | sort -u | cut -d'=' -f2 > ${TEMPORAL}/transcriptTmp.tab


    while read transcript; do fgrep ${transcript} ${CDS}/gencode.v27lift37.cds.${CHR}.gff3 | awk '{print $4,$5,$5-$4}' | awk -v transcript="$transcript" -v genes=${GENE} -v chr=${CHR}  '{sum+=$3} {printf $1","$2","} END{printf "\t"genes"\t"chr"\t"transcript"\t"sum"\n"}' | awk '{print $2,$3,$4,$5,$1}' | sed 's/,$//' | tr ' ' '\t' >> ${ANNOTATIONS}/cdsCoordinates.tab ;done < ${TEMPORAL}/transcriptTmp.tab
    
done

#### Check degenerancy by position

**Checking degenerancy by positions taking into account all transcripts and genes independently**  
Recoding CDS sequences to get 0fold, 2fold, 3fold and 4fold positions by transcript and genes, in order to estimate frequencies and divergence by type of functional sites.

In [ ]:
sys.path.insert(0, SRC)
from reverseComplement import reverseComplement
from degenerancy import degenerate

In [ ]:
dfGenes = pd.read_csv(DATA + '/annotations/gencodeGenesCleaned.tab',header = 0 ,usecols=['id','chr','strand'],sep='\t')
cds = pd.read_csv(DATA + '/annotations/cdsCoordinates.tab',header=0,sep='\t')
cds = pd.merge(cds, dfGenes[['id','strand','chr']],  how='right', left_on=['chr','id'], right_on = ['chr','id'])

In [ ]:
# Rewrite file each execution
degen = []
allPositions = []
nchr = []

import time
for index, row in cds.iterrows():
    start_time = time.time()
    print(index,row['id'])
    chrFile = px.Fasta('/home/jmurga/mkt/201903/rawData/humans/fastas/ref/' + row['chr'] +'.fa',sequence_always_upper=True)
    # Convert CDS list into numeric array
    coordinates = np.array(row['coordinates'].split(',')).astype(int).tolist()
    coordinates =  [coordinates[i:i+2] for i in range(0, len(coordinates), 2)]
    # Extract all CDS positions in a list in order to merge with degenerate sequences (same length -> same index)
    positions=[]
    for i in range(0,len(coordinates),1):
        positions.append(list(range(coordinates[i][0],coordinates[i][1]+1)))  
    positions = [item for sublist in positions for item in sublist]
    # Extract cds sequences
    seq = chrFile.get_spliced_seq(row['chr'], coordinates).seq

    if(row['strand'] == '-'):
        seq = reverseComplement(seq)
        allPositions = allPositions[::-1]
    if((len(seq)/3).is_integer() and seq[0:3]=='ATG'):
        m = degenerate(seq)
        degen.append(list(m))
        allPositions.append(positions)
        nchr.append([row['chr']] * len(m))
    print("--- %s seconds ---" % (time.time() - start_time))
#         degenerateDF = pd.DataFrame({'chr':j,'m':list(m),'POS':allPositions})
#         degeneratePositions = pd.concat([degeneratePositions,degenerateDF])
nchr = [item for sublist in nchr for item in sublist]
degen = [item for sublist in degen for item in sublist]
allPositions = [item for sublist in allPositions for item in sublist]

In [ ]:
df = pd.DataFrame({'CHR':nchr,'POS':allPositions,'degen':degen}) 
for i in df['CHR'].unique():
    tmp = df[df['CHR'] == i]
    tmp = tmp.groupby(['CHR','POS'])['degen'].apply(lambda x: ','.join(x)).reset_index()
    tmp.to_csv('/home/jmurga/mkt/201903/rawData/humans/annotations/degeneracyHumanPositions.tab',mode='a',index=False,sep='\t')

Cleaning positions degenerancy based on most constrain posibility

In [ ]:
degenerancyPositions = pd.read_csv(DATA + '/annotations/degeneracyHumanPositions.tab',sep='\t',header=0)
degenerancyPositions['type'] = np.nan

In [ ]:
def foldPositions(x):
    if('0' in x):
        return('0fold')
    elif('4' in x and '0' not in x and '2' not in x and '3' not in x):
        return('4fold')
    elif('2' in x and '0' not in x and '4' not in x and '3' not in x):
        return('2fold')
    elif('2' not in x and '0' not in x and '4' not in x) :
        return('3fold')
    else:
        return(x)

In [ ]:
degenerancyPositions['type'] = degenerancyPositions['m'].apply(foldPositions)

In [ ]:
df = degenerancyPositions[(degenerancyPositions['type']=='0fold') | (degenerancyPositions['type']=='4fold')]

In [ ]:
df.to_csv(DATA + '/annotations/zeroFourFoldPositions.tab',header=True,index=False,sep='\t')
df.to_hdf(DATA + '/annotations/zeroFourFoldPositions.h5','degenerancy')

### Creating Derived Allele Frequency and Divergence file by populations and type of site

#### Extracting mi and m0 position by largest transcript

In [ ]:
dfGenes = pd.read_csv(DATA + '/annotations/gencodeGenesCleaned.tab',header = 0 ,sep='\t')
cds = pd.read_csv(DATA + '/cdsCoordinates.tab',header = 0 ,sep='\t')
cds = cds.loc[cds.reset_index().groupby(['chr','id'])['transcriptSize'].idxmax()].reset_index(drop=True)
cds = pd.merge(dfGenes,cds,on=['chr','id'])
degeneratePositions = pd.read_hdf(DATA + '/annotations/zeroFourFoldPositions.h5','positions')

In [ ]:
for j in cds['chr'].unique():
    print(j)
    chrPositions = degeneratePositions[degeneratePositions['CHROM']==j]
#     for index, row in cds[cds['chr']==j].iterrows():
    for index, row in cds[cds['id']=='ENSG00000079102.16_3'].iterrows():
        print(index,row['id'])
        chrFile = Fasta(DATA + '/refFastas/' + row['chr'] +'.fa')
    #     Convert CDS list into numeric array
        coordinates = array(row['coordinates'].split(',')).astype(int).tolist()
        coordinates =  [coordinates[i:i+2] for i in range(0, len(coordinates), 2)]
        # Extract all CDS positions in a list in order to merge with degenerate sequences (same length -> same index)
        positions=[]
        for i in range(0,len(coordinates),1):
            positions.append(list(range(coordinates[i][0],coordinates[i][1]+1)))  
        allPositions = [item for sublist in positions for item in sublist]
        # Extract cds sequences
        seq = chrFile.get_spliced_seq(row['chr'].replace('chr',''), coordinates).seq.upper()
    #     print(len(seq))
        if(row['strand'] == '-'):
            seq = reverseComplement(seq)
            allPositions = allPositions[::-1]
        if((len(seq)/3).is_integer() and seq[0:3]=='ATG'):
            m = degenerate(seq)
            tmp = pd.DataFrame({'POS':allPositions,'m':list(m)})
            tmp['CHROM']=row['chr']
            tmp = pd.merge(chrPositions[['CHROM','POS','type']],tmp,on=['CHROM','POS'],how='right')
            counts = tmp['type'].value_counts()
            
            if(counts.shape[0]<2 or '4fold' not in counts.index):
                m0=0
                mi=counts['0fold']
            else:
                m0=counts['4fold']
                mi=counts['0fold']
            data = pd.DataFrame({'id':row['id'],'mi':mi,'m0':m0},index=[0])
            data.to_csv(DATA+'/annotations/refAnalizableSytes.tab',sep='\t',header=False,index=False,mode='a')
        else:
            data = pd.DataFrame({'id':row['id'],'mi':0,'m0':0},index=[0])
            data.to_csv(DATA+'/annotations/tmp/refAnalizableSytes.tab',sep='\t',header=False,index=False,mode='a')

In [ ]:
pops = ['ACB','ASW','BEB','CDX','CEU','CHB','CHS','CLM','ESN','FIN','GBR','GIH','GWD','IBS','ITU','JPT','KHV','LWK','MSL','MXL','PEL','PJL','PUR','STU','TSI','YRI']
refAnalizableSites = pd.read_csv(DATA+'/annotations/refAnalyzableSites.tab',sep='\t',header=None,names=['id','mi','m0'])

totalFoldPositionsByPop = pd.DataFrame()
for p in pops:
    print(p)
    refAnalizableSites['pop'] = p
    
    totalFoldPositionsByPop = totalFoldPositionsByPop.append(refAnalizableSites)

totalFoldPositionsByPop = totalFoldPositionsByPop.reset_index(drop=True)

#### Polymorphism

##### Manual recode

Extract coordinates from largest transcript to estimate allele frequency from variant data and merge with precalculated types of sites

In [ ]:
dfGenes = pd.read_csv(DATA + '/annotations/gencodeGenesCleaned.tab',header = 0 ,sep='\t')
cds = pd.read_csv(DATA + '/cdsCoordinates.tab',header = 0 ,sep='\t')
cds = cds.loc[cds.reset_index().groupby(['chr','id'])['transcriptSize'].idxmax()].reset_index(drop=True)

In [ ]:
cds.head()

Extracting largest transcript coordinates to check variants at these specific positions

In [ ]:
# columns = ['POS','id','transcript']
# for index,row in cds.iterrows():
#     print(index,row['id'])
#     coordinates = array(row['coordinates'].split(',')).astype(int).tolist()
#     coordinates =  [coordinates[i:i+2] for i in range(0, len(coordinates), 2)]
    
#     # Extract all CDS positions in a list in order to merge with degenerate sequences (same length -> same index)
#     positions=[]
    
#     for i in range(0,len(coordinates),1):
#         positions.append(list(range(coordinates[i][0],coordinates[i][1]+1)))  
#     allPositions = [item for sublist in positions for item in sublist]
    
# #     tmp = pd.DataFrame({'POS':allPositions,'id':row['id'],'transcript':row['transcript']})
#     tmp = pd.DataFrame({'CHROM':row['chr'],'POS':allPositions,'id':row['id'],'transcript':row['transcript']})
#     tmp.to_csv(DATA+'/annotations/largestTranscriptCoordinates.tab',mode='a',index=False,sep='\t')
#     tmp.to_hdf(DATA+'/annotations/largestTranscriptCoordinates.h5','positions',append=True)

In [ ]:
!python /home/jmurga/mkt/201903/scripts/src/sfsFromVcf.py --help

In [ ]:
!python /home/jmurga/mkt/201903/scripts/src/sfsFromVcf.py --data gencodeGenesCleaned.tab --cds largestTranscriptCoordinates.h5 --vcf 1000GP --chromosomes all --populations Phase3

!python /home/jmurga/mkt/201903/scripts/src/sfsFromVcf.py --data gencodeGenesCleaned.tab --cds cdsCoordinates.tab --vcf 1000GP --chromosomes all --populations Phase3

Subtract low quality positions from variants using bedtools intersect and 1000GP pilot mask

In [ ]:
intersectBed -a <(awk '{print $1"\t"$2-1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9"\t"$10}' manualRawFrequencies.tab) -b <(sed 's/chr//g' /data/shared/1000GP/Masks/20141020.pilot_mask.whole_genome.bed) | cut -f1,3,4,5,6,7,8,9,10 > manualRawFrequenciesMasked.tab

In [ ]:
rawFrequencies = pd.read(DATA + '/alleleFrequencies/manualRawFrequenciesMasked.tab',header=None,sep='\t')
rawFrequencies.columns = ['CHROM','POS','AC','AN','REF','ALT','AA','pop','id','transcript']
rawFrequencies.to_hdf(DATA + '/alleleFrequencies/manualRawFrequenciesMasked.h5','variants')

##### SFS, Pi and P0

In [ ]:
%time pol = pd.read_hdf('/home/jmurga/mkt/201903/rawData/humans/alleleFrequencies/manualRawFrequenciesMasked.h5') 
pol = pol[(pol['AC']!=0) & (pol['AC']!=pol['AN'])]
pol = pol[(pol['AA']!='.') & (pol['AA']!='N') & (pol['AA']!='-')].dropna()
# pol = pol[(pol['CHROM']==2)]

In [ ]:
%time pol['derivedAlleleFrequency'] = pol.apply(lambda x: x['AC']/x['AN'] if x['AA']==x['REF'] else (x['AN']-x['AC'])/x['AN'],axis=1)

In [ ]:
pol.head()

Checking positions

In [ ]:
# positions = pd.read_csv(DATA + '/annotations/zeroFourFoldPositions.tab',header=0,sep='\t')
%time positions = pd.read_hdf(DATA + '/annotations/zeroFourFoldPositions.h5','positions')
positions.columns = ['CHROM','POS','m','type']
positions['CHROM'] = positions['CHROM'].apply(lambda x: re.sub('chr','',x))

positions['CHROM']=positions['CHROM'].astype(str)
positions['POS']=positions['POS'].astype(int)

pol['CHROM']=pol['CHROM'].astype(str)
pol['POS']=pol['POS'].astype(int)

In [ ]:
pol = pd.merge(pol,positions,on=['CHROM','POS'],how='inner')
pol.head()

Binning by daf categories and creating new column from *type* to count number of occurrence by each category

In [ ]:
bins = np.arange(0,1.05,0.05)
labels = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1]
# daf['categories'] = pd.cut(daf['freq'],bins=bins,labels=labels).astype(float).fillna(0)
pol['categories'] = pd.cut(pol['derivedAlleleFrequency'],bins=bins,labels=labels)
# Creating new column from type to count number of occurrence by category
pol['count'] = pol['type']
pol.head()

Counting total of sites grouping 0fold and 4fold categories

In [ ]:
totalSites = pol.groupby(['id','CHROM','pop','type'])['categories'].count().reset_index()
totalSites = totalSites.pivot_table(index=['id','CHROM','pop'], columns=['type'],values='categories').reset_index()
totalSites.columns = ['id','chr','pop','pi','p0']
totalSites = totalSites.fillna(0)
totalSites.head()

Parsing Sites Frequency Spectrum to create a dataframe compatible with *iMKT R package*. Next steps include execute MKT over genes and populations using a modification of this packages. It was programmed to parse the SFS as string with dotComma separated values.

In [ ]:
sfs = pol.loc[:,['id','pop','type','categories','count']].groupby(['id','pop','type','categories']).count().reset_index()
sfs.columns = ['id','pop','type','categories','count']
sfs['count'] = sfs['count'].fillna(0).astype(int)
sfs = sfs.groupby(['id','pop','type'])['count'].apply(list).reset_index()
sfs['count'] = sfs['count'].apply(lambda x:';'.join(map(str,x)))
sfs = sfs.pivot_table(index=['id','pop'], columns=['type'],values='count',aggfunc=lambda x: ' '.join(x)).reset_index()
sfs.columns = ['id','pop','daf0f','daf4f']

In [ ]:
sfs.head()

In [ ]:
daf = pd.merge(totalSites,sfs,on=['id','pop'])
daf = daf.fillna(0)
print(daf.shape)
daf.head()

In [ ]:
daf[daf['id']=='ENSG00000115850.9_3']

#### Divergence

##### Extracting div sites from chimp alignment

In [ ]:
!python /home/jmurga/mkt/201903/scripts/src/alleleFrequencyByPop.py --help

In [ ]:
!python /home/jmurga/mkt/201903/scripts/src/alleleFrequencyByPop.py --data gencodeGenesCleaned.tab --cds largestTranscriptCoordinates.h5 --VCF Alns --chromosomes all 

Subtract low quality positions from variants using bedtools intersect and 1000GP pilot mask

In [ ]:
intersectBed -a <(awk '{print $1"\t"$2-1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"}' manualRawFrequencies.tab) -b <(sed 's/chr//g' /data/shared/1000GP/Masks/20141020.pilot_mask.whole_genome.bed) | cut -f1,3,4,5,6,7 > manualRawFrequenciesMasked.tab

In [ ]:
divPositions = pd.read(DATA + '/alleleFrequencies/manualRawFrequenciesMasked.tab',header=None,sep='\t')
divPositions.columns = ['CHROM','POS','REF','ALT','GT','id','transcript']
divPositions.to_hdf(DATA + '/alleleFrequencies/manualRawFrequenciesMasked.h5','variants')

##### Calculating div sites

In [ ]:
fixedPol = pd.read_hdf('/home/jmurga/mkt/201903/rawData/humans/alleleFrequencies/manualRawFrequenciesMasked.h5','variants')
fixedPol['AA'] = fixedPol['AA'].str.upper()
fixedPol = fixedPol[(fixedPol['AA']!='.') & (fixedPol['AA']!='N') & (fixedPol['AA']!='-')].dropna()
fixedPol = fixedPol[(fixedPol['AC']==0) | (fixedPol['AC']==fixedPol['AN'])]
fixedPol['CHROM'] = fixedPol['CHROM'].astype(str)
fixedPol['POS'] = fixedPol['POS'].astype(int)
fixedPol.head()

In [ ]:
div = pd.read_hdf('/home/jmurga/mkt/201903/rawData/humans/alleleFrequencies/manualDivergenceVariantsMasked.h5')
div = div[(div['ALT']!='N') & (div['ALT']!='')]
div['CHROM'] = div['CHROM'].astype(str)
div['POS'] = div['POS'].astype(int)
div.head()

In [ ]:
positions = pd.read_hdf(DATA + '/annotations/zeroFourFoldPositions.h5','positions')
positions.columns = ['CHROM','POS','m','type']
positions['CHROM'] = positions['CHROM'].astype(str)
positions['POS'] = positions['POS'].astype(int)
positions['CHROM'] = positions['CHROM'].apply(lambda x: re.sub('chr','',x))

In [ ]:
div = pd.merge(div,positions,on=['CHROM','POS'],how='inner')
div.head()

Iterate each pop and merge with divergent variants extract with VEP. Each iteration subset population and get only fixed variants in each population

In [ ]:
# Subset all populations in dataframe
pops = fixedPol['pop'].unique()

# Empty df to append each population
divSites = pd.DataFrame()

for p in pops:
    print(p)
    # Subset population and fixed variants
    fixedPolSubset = fixedPol[fixedPol['pop'] == p]

    # Merging all fixed variants and posible divergent sites
    divPol = pd.merge(div[['POS','CHROM','GT','id','type']],fixedPolSubset[['POS','CHROM','AC']],on=['CHROM','POS'],how='outer')
    
    # Extract and count real divergent sites     
    divPol.loc[:,'d'] = np.nan
    divPol.loc[:,'d'] = divPol[~divPol['GT'].isna()]['d'].fillna(1)
    divPol = divPol[~divPol['d'].isna()]
    
    # Cleaning and grouping divergent site by id
    tmp = divPol[['POS','id','type','d']]
    tmp = tmp.groupby(['id','type'])['d'].count().reset_index()
    tmp = tmp.pivot_table(index=['id'],columns=['type'],values='d').reset_index()
    tmp.columns = ['id','di','d0']
    tmp = tmp.fillna(0)
    tmp['pop'] = p

    # Append results to df by pop
    divSites = divSites.append(tmp)

In [ ]:
divSites[divSites['id']=='ENSG00000115850.9_3']

#### Merging Derived Allele Frequencies, divergency and total analyzed sites

In [ ]:
dfGenes = pd.read_csv(DATA + '/annotations/gencodeGenesCleaned.tab',header = 0 ,usecols=['id','chr'],sep='\t') 
dfGenes['chr']=dfGenes['chr'].apply(lambda x: re.sub('chr','',x))
pops = ['ACB','ASW','BEB','CDX','CEU','CHB','CHS','CLM','ESN','FIN','GBR','GIH','GWD','IBS','ITU','JPT','KHV','LWK','MSL','MXL','PEL','PJL','PUR','STU','TSI','YRI']

ids = pd.DataFrame()
for p in pops:
    print(p)
    dfGenes['pop'] = p
    
    ids = ids.append(dfGenes)

ids = ids.reset_index(drop=True)

In [ ]:
dfToMerge = pd.merge(daf,totalFoldPositionsByPop,on=['id','pop'],how='outer')
dfToMerge['id'].shape

In [ ]:
dfToMerge = pd.merge(dfToMerge,ids,on=['chr','id','pop'],how='right')

In [ ]:
humanGenes = pd.merge(dfToMerge, divSites,on=['id','pop'],how='outer')
humanGenes = humanGenes.sort_values(['chr','id','pop'])
humanGenes[humanGenes['daf0f'].isna()].loc[:,'daf0f'] = '0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0'
humanGenes[humanGenes['daf4f'].isna()].loc[:,'daf4f'] = '0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0'
humanGenes['daf0f'] = humanGenes['daf0f'].fillna('0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0')
humanGenes['daf4f'] = humanGenes['daf4f'].fillna('0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0')
humanGenes = humanGenes.fillna(0)
humanGenes.shape

In [ ]:
dfGenes = pd.read_csv(DATA + '/annotations/gencodeGenesCleaned.tab',header = 0 ,usecols=['id','name'],sep='\t')
humanGenes = pd.merge(humanGenes,dfGenes,on=['id'])

In [ ]:
humanGenes.head()

In [ ]:
dfRecomb = pd.read_csv(DATA + '/genesRecomb.tab',header = 0 ,sep='\t')
humanGenes = pd.merge(humanGenes,dfRecomb,on=['id'],how='left')

In [ ]:
humanGenes.head(10)

In [ ]:
humanGenes.to_csv('/home/jmurga/mkt/201903/rawData/humans/humanGenesPhase3Masked.tab',sep='\t',index=False,header=True,na_rep="NA")